## Planning production and inventories

In this notebook we consider a planning problem that covers $T$ periods of time.
If there is a production in a given period $t \in \{1,\dots,T\}$, a fixed cost $c^{\text{fixed}}_{t}$ is incurred regardless of the production amount.
We assume that the production demand $d_{t}$ at a period $t$ is known, and it has to be met.
However, we allow *overproduction* where the excess goods are stored for the demand of the later periods.
This inflicts a *holding cost* $c^{\text{holding}}_{t}$ per unit stored.

## Problem formulation

| **Parameters** | |
|:---|:---|
| $T$ | The production time horizon. |
| $t = 1,\dots,T$ | The production period. |
| $c_{t}$ | The production cost per unit produced. |
| $c^{\text{fixed}}_{t}$ | The fixed cost when there is a production at period $t$. |
| $c^{\text{holding}}_{t}$ | The holding cost per unit stored. |
| $d_{t}$ | The demand at period $t$. |
| $M_{t}$ | The production capacity. |
| **Decision** |
| $x_{t}$ | Units of production during period $t$. |
| $s_{t}$ | Units of products to be stored during period $t$. |
| $y_{t} \in \{0,1\}$ | Binary variable that is $1$ if there is a production and $0$ if there is no production. |

### Constraints

- Demand must be met during each period: $x_{t} - s_{t} + s_{t-1} = d_{t}$
- The production/non-production decision: $x_{t} \leq M_{t}y_{t}$
- Storing amount must not exceed the produced amount: $s_{t} \leq x_{t}$
- Non-negativity: $x_{t},s_{t} \geq 0$

### Objective function

To minimize the total cost
$$
Z = \sum_{t=1}^{T} [ c_{t}x_{t} + c^{\text{holding}}_{t}s_{t} + c^{\text{fixed}}_{t}y_{t} ] \ \leftarrow \text{Minimize}
$$

---

## Question 1


Suppose that the production is planned in for 4 weeks, and $c_{t}, c_{t}^{\text{fixed}}, c_{t}^{\text{holding}}$ are the weekly costs.
Try to model the problem so that the demand is increasing weekly, and the holding cost $c_{t}^{\text{holding}}$ is much cheaper than the production cost $c_{t+1}$.

In [ ]:
!pip install pyscipopt

In [ ]:
from pyscipopt import Model, quicksum

In [ ]:
# Input data
T = 4 # time horizon
Times = range(T)
c = np.array([5, 4, 6, 5]) # per-unit production cost
c_fixed = 20 # fixed cost (assumed to be equal for all times)
c_holding = np.array([1, 0.5, 0.7, 0.2]) # holding cost
d = np.array([40, 50, 60, 70]) # demand
M = 65 # capacity

# Create the model
plan = Model()

# Add variables
x = [plan.addVar(vtype='C', lb=0) for t in Times]
s = [plan.addVar(vtype='C', lb=0) for t in Times]
y = [plan.addVar(vtype='B') for t in Times]

# Add constraints
for t in Times:
    plan.addCons(x[t] <= M*y[t])
    plan.addCons(s[t] <= x[t])
plan.addCons(x[0] - s[0] == d[0])
for t in range(1,T):
    plan.addCons(x[t] - s[t] + s[t-1] == d[t])

# Set objective
plan.setObjective(quicksum(c[t]*x[t] + c_holding[t]*s[t] + c_fixed*y[t] for t in Times))

# Solve and display solutions
plan.hideOutput()
plan.optimize()
SOL = plan.getBestSol()
x = np.array([SOL[x[t]] for t in Times])
s = np.array([SOL[s[t]] for t in Times])
for t in Times: print(f"Week {t}: Production = {x[t]} units. Units to keep in inventory = {s[t]} units")

## Question 2

Modify the holding cost and production cost so that $c_{t}^{\text{holding}} >> c_{t+1}$.
What impact it brings to the optimal solution ?

In [ ]:
# Input data
T = 4 # time horizon
Times = range(T)
c = np.array([5, 4, 6, 5]) # per-unit production cost
c_fixed = 20 # fixed cost (assumed to be equal for all times)
c_holding = np.array([15, 12, 15, 12]) # holding cost
d = np.array([40, 50, 60, 70]) # demand
M = 65 # capacity

# Create the model
plan = Model()

# Add variables
x = [plan.addVar(vtype='C', lb=0) for t in Times]
s = [plan.addVar(vtype='C', lb=0) for t in Times]
y = [plan.addVar(vtype='B') for t in Times]

# Add constraints
for t in Times:
    plan.addCons(x[t] <= M*y[t])
    plan.addCons(s[t] <= x[t])
plan.addCons(x[0] - s[0] == d[0])
for t in range(1,T):
    plan.addCons(x[t] - s[t] + s[t-1] == d[t])

# Set objective
plan.setObjective(quicksum(c[t]*x[t] + c_holding[t]*s[t] + c_fixed*y[t] for t in Times))

# Solve and display solutions
plan.hideOutput()
plan.optimize()
SOL = plan.getBestSol()
x = np.array([SOL[x[t]] for t in Times])
s = np.array([SOL[s[t]] for t in Times])
for t in Times: print(f"Week {t}: Production = {x[t]} units. Units to keep in inventory = {s[t]} units")

----